# Using the Chinook Database to Answer Hypothetical Business Questions with SQLite

In this notebook, I will be demonstrating some intermediate and complex SQL queries which can answer hypothetical business questions using the Chinook sample database.

The Chinook sample database covers data from a hypothetical record store. There are 11 tables:

- employees - stores employee data
- customers - stores customer data
- invoices & invoice_items - 2 tables storing invoice data
- artists - stores artists data for records sold in the store
- albums - stores albums data for records sold in the store
- media_types - stores media types such as MPEG Audio and AAC audio files
- genres - stores music genres such as rock, jazz, hip-hop etc.
- tracks - stores songs data
- playlists & playlists_track - stores data about playlists and tracks within playlists
- country_data - stores country data of tracks

I have uploaded the database scheme to the repo for reference.

## Connecting the notebook to the database file

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

## Getting familiar with the data

First, lets write a query to find the tables in the database.

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type in ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


Now, lets write a few quick queries to get a sense of some of the tables and the data.

In [3]:
%%sql
SELECT * FROM album LIMIT 5;

 * sqlite:///chinook.db
Done.


album_id,title,artist_id
1,For Those About To Rock We Salute You,1
2,Balls to the Wall,2
3,Restless and Wild,2
4,Let There Be Rock,1
5,Big Ones,3


In [4]:
%%sql
SELECT * FROM artist LIMIT 5;

 * sqlite:///chinook.db
Done.


artist_id,name
1,AC/DC
2,Accept
3,Aerosmith
4,Alanis Morissette
5,Alice In Chains


In [5]:
%%sql
SELECT * FROM employee LIMIT 5;

 * sqlite:///chinook.db
Done.


employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
1,Adams,Andrew,General Manager,None,1962-02-18 00:00:00,2016-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
2,Edwards,Nancy,Sales Manager,1,1958-12-08 00:00:00,2016-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2017-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
4,Park,Margaret,Sales Support Agent,2,1947-09-19 00:00:00,2017-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
5,Johnson,Steve,Sales Support Agent,2,1965-03-03 00:00:00,2017-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com


# 1. Finding which genres sell the most tracks in the USA

For the first hypothetical business question, we will imagine the record store has just signed a deal with a record label and we have been tasked with choosing 3 albums from 4 that will be added to the store. We have the genres of the 4 albums:

- Hip-hop
- Punk
- Pop
- Blues

The record label specialises in artists from the USA, and so have given the record store money to advertise in the USA.

We need to find out *which genres sell the best in the USA, so we can maximise the advertising profits, and make a recommendation on which albums the store should purchase*.

The query for this is as follows.

In [6]:
%%sql
WITH total AS
    (
    SELECT COUNT(*) AS total FROM invoice
    WHERE billing_country = 'USA'
    )
SELECT
    genre.name genre,
    COUNT(genre.name) usa_genre_sale_count,
    ROUND(CAST(COUNT(genre.name) AS FLOAT) / total.total, 2) usa_genre_pct
FROM invoice, total
INNER JOIN invoice_line il ON il.invoice_line_id = invoice.invoice_id
INNER JOIN track t ON t.track_id = il.track_id
INNER JOIN genre ON genre.genre_id = t.genre_id
WHERE invoice.billing_country = "USA"
GROUP BY genre.name
ORDER BY usa_genre_pct DESC;

 * sqlite:///chinook.db
Done.


genre,usa_genre_sale_count,usa_genre_pct
Rock,88,0.67
Alternative & Punk,10,0.08
Metal,8,0.06
R&B/Soul,6,0.05
Classical,4,0.03
Pop,2,0.02
Latin,2,0.02
Jazz,3,0.02
Electronica/Dance,2,0.02
Alternative,2,0.02


The table illustrates that within the USA, more than half of the total sales of tracks fall under the Rock genre.

With regards to the albums provided by the record label the store should purchase the albums falling under the punk, pop and blues genres as hip-hop doesn't seem to be bought in the USA stores.

# 2. Analysing employee sales performance

In order to visualise employee sales performance, we will write a quey that finds the total dollar amount of sales assigned to each sales support agent.

We will also include the hire date of the employees for context.

In [7]:
%%sql
SELECT
    e.first_name || " " || e.last_name employee_name,
    e.hire_date,
    ROUND(SUM(i.total), 2) total_dollar_sales
FROM employee e
INNER JOIN customer c ON c.support_rep_id = e.employee_id
INNER JOIN invoice i ON i.customer_id = c.customer_id
WHERE e.title = "Sales Support Agent"
GROUP BY employee_name
ORDER BY total_dollar_sales DESC;

 * sqlite:///chinook.db
Done.


employee_name,hire_date,total_dollar_sales
Jane Peacock,2017-04-01 00:00:00,1731.51
Margaret Park,2017-05-03 00:00:00,1584.0
Steve Johnson,2017-10-17 00:00:00,1393.92


Jane Peacock is the most successful Sales Support Agent. The variance in sales might not be indicative of performance however and could also be dependent on how long the employee has been at the company, shown by the negative correlation between hire_date and total_dollar_sales. 

# 3. Analysing Country Sales Data

For the next business analysis task, we will be analysing the sales data for customers from each country.

We will calculate data for each country on the
- total number of customers
- total value of sales
- average value of sales per customer
- average order value

To do this we will create a view of unsorted country data from which we will query from. This complex query is detailed below:

In [8]:
%%sql
CREATE VIEW country_data_unsorted AS
    WITH
        country_customers AS
        (
        SELECT
            country,
            COUNT(customer_id) no_of_customers
        FROM customer
        GROUP BY 1
        ),
    
        country_total_sales AS
        (
        SELECT
            customer.country,
            SUM(invoice.total) total_sales
        FROM customer
        INNER JOIN invoice ON invoice.customer_id = customer.customer_id
        GROUP BY 1
        ),
    
        country_avg_sales AS
        (
        SELECT
            customer.country,
            AVG(invoice.total) avg_sales
        FROM customer
        INNER JOIN invoice ON invoice.customer_id = customer.customer_id
        GROUP BY 1
        ),
        country_unsorted AS
        (
        SELECT
            cc.country,
            cc.no_of_customers,
            ROUND(cts.total_sales,2) total_sales,
            ROUND(cas.avg_sales,2) avg_sales,
            ROUND(cas.avg_sales / cc.no_of_customers,2) avg_sale_per_customer
        FROM country_customers cc
        INNER JOIN country_total_sales cts ON cts.country = cc.country
        INNER JOIN country_avg_sales cas ON cas.country = cts.country
        ORDER BY cc.no_of_customers DESC
        )
SELECT
    CASE
        WHEN no_of_customers = 1 THEN "Other"
        ELSE country
    END AS country,
    SUM(no_of_customers) no_of_customers,
    total_sales,
    avg_sales,
    avg_sale_per_customer
FROM country_unsorted
GROUP BY 1;


 * sqlite:///chinook.db
(sqlite3.OperationalError) table country_data_unsorted already exists
[SQL: CREATE VIEW country_data_unsorted AS
    WITH
        country_customers AS
        (
        SELECT
            country,
            COUNT(customer_id) no_of_customers
        FROM customer
        GROUP BY 1
        ),
    
        country_total_sales AS
        (
        SELECT
            customer.country,
            SUM(invoice.total) total_sales
        FROM customer
        INNER JOIN invoice ON invoice.customer_id = customer.customer_id
        GROUP BY 1
        ),
    
        country_avg_sales AS
        (
        SELECT
            customer.country,
            AVG(invoice.total) avg_sales
        FROM customer
        INNER JOIN invoice ON invoice.customer_id = customer.customer_id
        GROUP BY 1
        ),
        country_unsorted AS
        (
        SELECT
            cc.country,
            cc.no_of_customers,
            ROUND(cts.total_sales,2) total_sales,
       

In [9]:
%%sql
SELECT
    country,
    no_of_customers,
    total_sales,
    avg_sales,
    avg_sale_per_customer
FROM
    (
    SELECT
        country_data_unsorted.*,
        CASE
            WHEN country_data_unsorted.country = "Other" THEN 1
        END AS sort
    FROM country_data_unsorted
    )
ORDER BY sort ASC, total_sales DESC;

 * sqlite:///chinook.db
Done.


country,no_of_customers,total_sales,avg_sales,avg_sale_per_customer
USA,13,1040.49,7.94,0.61
Canada,8,535.59,7.05,0.88
Brazil,5,427.68,7.01,1.4
France,5,389.07,7.78,1.56
Germany,4,334.62,8.16,2.04
Czech Republic,2,273.24,9.11,4.55
United Kingdom,3,245.52,8.77,2.92
Portugal,2,185.13,6.38,3.19
India,2,183.15,8.72,4.36
Other,15,39.6,7.92,7.92


Clearly, the USA is the most profitable market. From the *avg_sales* column though there may be opportunity within the Czech Republic, United Kingdom and India markets.